In [12]:
import os

In [13]:
%pwd

'e:\\Data Science\\LoanApprovalAppWith-Mlflow\\Loan-APP'

In [3]:
os.chdir("../")

In [4]:
%pwd

'e:\\Data Science\\LoanApprovalAppWith-Mlflow\\Loan-APP'

In [14]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/shubh-2291/shubh-2291-Loan_Approval_App_With_Mlflow.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "shubh-2291"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "bda8667039f829190ef0956963cf5992817e71f0"

In [15]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str
    

In [16]:
from Loan_APP.constants import *
from Loan_APP.utils.common import read_yaml, create_directories, save_json

In [17]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.RandomForestClassifier
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path= config.test_data_path,
            model_path= config.model_path,
            all_params= params,
            metric_file_name= config.metric_file_name,
            target_column= schema.name,
            mlflow_uri="https://dagshub.com/shubh-2291/shubh-2291-Loan_Approval_App_With_Mlflow.mlflow"
        )
        
        return model_evaluation_config
        

In [18]:
import os
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score 
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [19]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
        
    def eval_metric(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        
        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[self.config.target_column]
        
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_uri_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            predicted_value = model.predict(test_x)
            model_score = model.score(test_x, test_y)
            oob_score = model.oob_score_
            # confusion_matrix = confusion_matrix(test_y, predicted_value)
            scores = {"model_score":model_score, "oob_score": oob_score}
            save_json(path= Path(self.config.metric_file_name), data=scores)
            
            mlflow.log_params(self.config.all_params)
            
            mlflow.log_metric("model_score",model_score )
            mlflow.log_metric("oob_score",oob_score )
            # mlflow.log_metric("confusion_matrix",confusion_matrix)
            
            if tracking_uri_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name= "RandomForestModel")
            else:
                mlflow.sklearn.log_model(model, "model")

In [20]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config= model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2025-04-02 20:26:01,321: INFO: common: yaml file: config\config.yaml is loaded successfully!]
[2025-04-02 20:26:01,331: INFO: common: yaml file: params.yaml is loaded successfully!]
[2025-04-02 20:26:01,339: INFO: common: yaml file: schema.yaml is loaded successfully!]
[2025-04-02 20:26:01,344: INFO: common: created directory at: artifacts]
[2025-04-02 20:26:01,347: INFO: common: created directory at: artifacts/model_evaluation]
[2025-04-02 20:26:01,947: WARNING: connectionpool: Retrying (Retry(total=6, connect=7, read=6, redirect=7, status=7)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /shubh-2291/shubh-2291-Loan_Approval_App_With_Mlflow.mlflow/api/2.0/mlflow/runs/create]
[2025-04-02 20:26:03,949: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


2025/04/02 20:26:16 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'RandomForestModel' already exists. Creating a new version of this model...
2025/04/02 20:26:26 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: RandomForestModel, version 3
Created version '3' of model 'RandomForestModel'.


🏃 View run selective-duck-26 at: https://dagshub.com/shubh-2291/shubh-2291-Loan_Approval_App_With_Mlflow.mlflow/#/experiments/0/runs/a8a3a9922b7a4ec9bcd9ecdaec9d0cc2
🧪 View experiment at: https://dagshub.com/shubh-2291/shubh-2291-Loan_Approval_App_With_Mlflow.mlflow/#/experiments/0
